In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.9.1
torchvision version: 0.24.1


In [2]:
import os
from pathlib import Path

# المسار الحالي
cwd = Path.cwd()

print(f"📂 محتويات المجلد: {cwd}")
print("-" * 30)

# طباعة قائمة بكل الملفات والمجلدات الموجودة هنا
for item in os.listdir(cwd):
    print(item)

print("-" * 30)

📂 محتويات المجلد: /Users/abdrhmn/myproject/0-deeplearning-pytorch/deep-learning-by-pytorch
------------------------------
cv.ipynb
pytorch_custom_datasets.ipynb
05_pytorch_going_modular_cell_mode.ipynb
.DS_Store
helper_functions.py
test
05_pytorch_going_modular_cell_mode_video.ipynb
test1
07_pytorch_expirment_tracking.ipynb
models
__pycache__
README.md
helper_utils.py
.gitignore
1.ipynb
.venv
.gitattributes
05_pytorch_going_modular.md
pytorch_datasets_.ipynb
going_modular
.git
.vscode
06_transfer_learning.ipynb
data
gmod.zip
0-pytorch.ipynb
pytorch-deep-learning
0.ipynb
------------------------------


In [3]:
import os
import requests
from pathlib import Path

# 1. تحديد مسار المجلد going_modular
target_directory = Path.cwd() / "going_modular"

# 2. إنشاء المجلد إذا لم يكن موجوداً
if not target_directory.exists():
    target_directory.mkdir(parents=True, exist_ok=True)
    print(f"[INFO] Created directory: {target_directory}")

# 3. تحديد الملفات التي نحتاجها من كورس Daniel Bourke
files_to_download = {
    "data_setup.py": "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/data_setup.py",
    "engine.py": "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/engine.py",
    "model_builder.py": "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/model_builder.py",
    "utils.py": "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/utils.py" 
}

# 4. تحميل الملفات
for filename, url in files_to_download.items():
    file_path = target_directory / filename
    
    # تحميل الملف فقط إذا لم يكن موجوداً
    if not file_path.exists():
        print(f"[INFO] Downloading {filename}...")
        response = requests.get(url)
        with open(file_path, "wb") as f:
            f.write(response.content)
    else:
        print(f"[INFO] {filename} already exists.")

# 5. إنشاء ملف __init__.py فارغ لضمان أن بايثون يتعامل معه كـ package
(target_directory / "__init__.py").touch()

print("\n[SUCCESS] تم تجهيز مجلد going_modular بنجاح! ✅")
print("الآن يمكنك تجربة تشغيل كود الـ import مرة أخرى.")

[INFO] data_setup.py already exists.
[INFO] engine.py already exists.
[INFO] model_builder.py already exists.
[INFO] utils.py already exists.

[SUCCESS] تم تجهيز مجلد going_modular بنجاح! ✅
الآن يمكنك تجربة تشغيل كود الـ import مرة أخرى.


In [4]:
import sys
from pathlib import Path

# الحصول على المسار الحالي الذي يعمل فيه النوت بوك
current_dir = Path.cwd()

# طباعة المسار للتأكد من مكانك الحالي
print(f"Current Directory: {current_dir}")

# إذا كان النوت بوك داخل مجلد models، نحتاج للرجوع خطوة للوراء (parent)
# للوصول إلى المجلد deep-learning-by-pytorch حيث يوجد going_modular
if current_dir.name == "models":
    root_path = current_dir.parent
else:
    # احتياطياً: افترض أنك في المسار الصحيح أو عدل هذا السطر حسب مسارك
    root_path = current_dir

# إضافة المسار الرئيسي إلى بايثون
sys.path.append(str(root_path))

print(f"Added to sys.path: {root_path}")

# --- الآن جرب الاستيراد (تأكد من تصحيح الكلمة إلى modular) ---
try:
    from going_modular import data_setup, engine
    print("Import successful! ✅")
except ImportError as e:
    print(f"Import failed: {e} ❌")
    print("تأكد أن مجلد going_modular موجود داخل المسار الذي تمت طباعته أعلاه.")

Current Directory: /Users/abdrhmn/myproject/0-deeplearning-pytorch/deep-learning-by-pytorch
Added to sys.path: /Users/abdrhmn/myproject/0-deeplearning-pytorch/deep-learning-by-pytorch
Import successful! ✅


In [5]:
# Continue with regular imports
%matplotlib inline
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
fatal: destination path 'pytorch-deep-learning' already exists and is not an empty directory.
mv: pytorch-deep-learning/going_modular: No such file or directory
rm: pytorch-deep-learning/.git/objects/pack/pack-a74a8c228cbf1f469f43bcde8ba08dbed73cd4c3.pack: Operation not permitted
rm: pytorch-deep-learning/.git/objects/pack/pack-a74a8c228cbf1f469f43bcde8ba08dbed73cd4c3.idx: Operation not permitted
rm: pytorch-deep-learning/.git/objects/pack/pack-a74a8c228cbf1f469f43bcde8ba08dbed73cd4c3.rev: Operation not permitted
rm: pytorch-deep-learning/.git/objects/pack: Directory not empty
rm: pytorch-deep-learning/.git/objects: Directory not empty
rm: pytorch-deep-learning/.git: Directory not empty
rm: pytorch-deep-learning: Directory not empty


In [6]:
from going_modular import data_setup, engine

print("Import worked successfully!")

Import worked successfully!


In [7]:
def set_seed (seed:int=42):
    import random
    import numpy as np
    import torch

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [8]:
import os 
import zipfile
from pathlib import Path 
import requests 
def download_data  (source: str,destination:str,remove_source:bool=True):
    """Download and unzip data from a given URL.

    Args:
        source (str): URL of the zip file to download.
        destination (str): Path to the directory where the data should be extracted.
        remove_source (bool, optional): Whether to delete the zip file after extraction. Defaults to True.
    """
    # Download the file from `source` and save it locally under `destination`:
    data_path=Path("data/")
    image_path=data_path/destination
    if image_path.is_dir():
        print(f"[INFO] {destination} already exists. Skipping download.")
    else:
        print(f"[INFO] Downloading {source} to {destination}...")
        image_path.mkdir(parents=True,exist_ok=True)
        target_file=Path(source).name
        with open (data_path/target_file, "wb")as f:
            request=requests.get(source)
            print(f"[INFO] Downloading {target_file}...")
            f.write(request.content)
        # Unzip the file
        print(f"[INFO] Unzipping {target_file}...")
        with zipfile.ZipFile(data_path/target_file, 'r') as zip_ref:
            zip_ref.extractall(image_path)
    # Remove the zip file
        if remove_source:
            os.remove(data_path/target_file)
            print(f"[INFO] Removed {target_file}.")
            
    return image_path
            
    

In [9]:
# اختر الكيرنال "Python (.venv)" من Kernel → Change Kernel
%matplotlib inline
import matplotlib.pyplot as plt
import requests

print("Everything works ✔️")


Everything works ✔️


In [10]:
image_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                           destination="pizza_steak_sushi")
image_path

[INFO] pizza_steak_sushi already exists. Skipping download.


PosixPath('data/pizza_steak_sushi')

In [11]:
train_dir=image_path/"train"
test_die= image_path/"test"
train_dir, test_die

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [12]:
# set up imagenet normalization values
from torchvision import transforms
normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
manual_transforms=transforms.Compose([
    transforms.Resize((224,224)),transforms.ToTensor(),normalize])

In [13]:
import os
os.getcwd()


'/Users/abdrhmn/myproject/0-deeplearning-pytorch/deep-learning-by-pytorch'

In [14]:
from going_modular import engine,utils,model_builder,data_setup
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader 
train_dataloader,test_dataloader,class_names=data_setup.create_dataloaders(train_dir=train_dir,
                                                                        test_dir=test_die,transform=manual_transforms,batch_size=32)

In [15]:
import torch

# 1. تعريف الـ device بشكل ديناميكي
device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"

print(f"Using device: {device}")

Using device: mps


In [16]:

train_dir= image_path/"train"
test_dir=image_path/"test"
train_dir, test_dir
# setup  pretrained model
import torchvision
wights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
# get transforms form wights
automatic_transforms=wights.transforms()
# create Dataloaders
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                            test_dir=test_dir,
                                                                            transform=automatic_transforms,
                                                                            batch_size=32)
train_dataloader,test_dataloader,class_names,len(class_names)

(<torch.utils.data.dataloader.DataLoader at 0x135cffeb0>,
 ['pizza', 'steak', 'sushi'],
 3)

In [17]:
# Get a pretrained model
import torchvision
#model= torchvision.models.efficientnet_b0(pretrained=True).to(device)
wights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
model= torchvision.models.efficientnet_b0(weights=wights).to(device)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [18]:
# Freeze the base model layers
for pram in model.features.parameters():
    pram.requires_grad=False


In [19]:
set_seed(42)
model.classifier=nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True)
).to(device)
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [20]:
from torchinfo import summary 
summary(model, input_size=(32, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"],col_width=20,row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [21]:
# Train  a single model 
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
set_seed(42)

 

In [22]:
# علامة التعجب مهمة لتشغيل أوامر التيرمينال من داخل النوت بوك
!pip install tensorboard

In [23]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
writer= SummaryWriter(f"runs/efficientnet_experiment_1_{datetime.now().strftime('%Y%m%d-%H%M%S')}")
